In [ ]:
# 安裝必要的套件
!pip install librosa soundfile ipython

# 安裝額外的語速調整套件 (可選，提供更好的音質)
!pip install pyrb

# 如果上面的套件安裝失敗，可以嘗試：
# !pip install psola
# !pip install pydub

In [2]:
import librosa
import soundfile as sf
import numpy as np
from IPython.display import Audio, display
import matplotlib.pyplot as plt

# 方法1: 使用 librosa 的 STFT 和 phase vocoder (推薦，保持音調不變)
def adjust_audio_speed_librosa(input_file, speed_factor=1.0, output_file=None):
    """
    使用 librosa 的 phase vocoder 調整語速（保持音調不變）
    
    參數:
    - input_file: 輸入音檔路徑
    - speed_factor: 語速倍率 (1.0=原速, 2.0=兩倍速, 0.5=半速)
    - output_file: 輸出音檔路徑 (可選)
    
    回傳:
    - (audio_data, sample_rate): 處理後的音檔數據和採樣率
    """
    # 載入音檔
    y, sr = librosa.load(input_file)
    
    # 使用 STFT 和 phase vocoder 調整語速
    # 計算 STFT
    stft = librosa.stft(y)
    
    # 使用 phase vocoder 調整時間
    stft_stretched = librosa.phase_vocoder(stft, rate=speed_factor)
    
    # 反變換回時域
    y_stretched = librosa.istft(stft_stretched)
    
    # 如果指定輸出檔案，則儲存
    if output_file:
        sf.write(output_file, y_stretched, sr)
    
    return y_stretched, sr

# 方法2: 使用 pyrb 庫 (WSOLA 算法，音質較好)
def adjust_audio_speed_pyrb(input_file, speed_factor=1.0, output_file=None):
    """
    使用 pyrb (WSOLA算法) 調整語速（保持音調不變）
    需要先安裝: pip install pyrb
    
    參數:
    - input_file: 輸入音檔路徑
    - speed_factor: 語速倍率 (1.0=原速, 2.0=兩倍速, 0.5=半速)
    - output_file: 輸出音檔路徑 (可選)
    
    回傳:
    - (audio_data, sample_rate): 處理後的音檔數據和採樣率
    """
    try:
        import pyrb
        
        # 載入音檔
        y, sr = librosa.load(input_file)
        
        # pyrb 的正確用法 - 使用 stretch 函數
        if hasattr(pyrb, 'time_stretch'):
            y_stretched = pyrb.time_stretch(y, sr, speed_factor)
        elif hasattr(pyrb, 'stretch'):
            y_stretched = pyrb.stretch(y, sr, speed_factor)
        else:
            print("pyrb API 不相容，請嘗試其他方法")
            return None, None
        
        # 如果指定輸出檔案，則儲存
        if output_file:
            sf.write(output_file, y_stretched, sr)
        
        return y_stretched, sr
    
    except ImportError:
        print("pyrb 未安裝，請執行: pip install pyrb")
        return None, None
    except Exception as e:
        print(f"pyrb 處理失敗: {e}")
        print("請嘗試使用 librosa 方法")
        return None, None

# 方法3: 使用 librosa 的簡化版本 (適用於較新版本)
def adjust_audio_speed_simple(input_file, speed_factor=1.0, output_file=None):
    """
    使用 librosa 的簡化方法調整語速
    
    參數:
    - input_file: 輸入音檔路徑  
    - speed_factor: 語速倍率 (1.0=原速, 2.0=兩倍速, 0.5=半速)
    - output_file: 輸出音檔路徑 (可選)
    
    回傳:
    - (audio_data, sample_rate): 處理後的音檔數據和採樣率
    """
    # 載入音檔
    y, sr = librosa.load(input_file)
    
    # 使用 librosa 的時間拉伸功能
    try:
        # 嘗試使用新版本的 API
        y_stretched = librosa.effects.time_stretch(y, rate=speed_factor)
    except:
        # 如果新版本不支援，使用 STFT 方法
        stft = librosa.stft(y)
        stft_stretched = librosa.phase_vocoder(stft, rate=speed_factor)
        y_stretched = librosa.istft(stft_stretched)
    
    # 如果指定輸出檔案，則儲存
    if output_file:
        sf.write(output_file, y_stretched, sr)
    
    return y_stretched, sr

# 建議使用的主要函數
def adjust_audio_speed(input_file, speed_factor=1.0, output_file=None, method='librosa'):
    """
    主要的語速調整函數
    
    參數:
    - input_file: 輸入音檔路徑
    - speed_factor: 語速倍率 (1.0=原速, 2.0=兩倍速, 0.5=半速)
    - output_file: 輸出音檔路徑 (可選)
    - method: 'librosa' 或 'pyrb'
    
    回傳:
    - (audio_data, sample_rate): 處理後的音檔數據和採樣率
    """
    if method == 'pyrb':
        return adjust_audio_speed_pyrb(input_file, speed_factor, output_file)
    else:
        return adjust_audio_speed_librosa(input_file, speed_factor, output_file)

# 創建測試音檔的函數
def create_test_audio(filename='test_audio.wav', duration=3, freq=440):
    """創建一個測試音檔"""
    sr = 22050
    t = np.linspace(0, duration, int(sr * duration))
    # 創建一個包含多個頻率的和聲
    y = (np.sin(2 * np.pi * freq * t) + 
         0.5 * np.sin(2 * np.pi * freq * 2 * t) + 
         0.3 * np.sin(2 * np.pi * freq * 3 * t))
    y = y * 0.2  # 降低音量
    sf.write(filename, y, sr)
    return filename, sr

# 使用範例
print("=== 語速調整工具 ===")
print("支援的方法:")
print("1. librosa (使用 phase vocoder) - 推薦")
print("2. pyrb (使用 WSOLA 算法) - 可選，需要正確安裝")
print()

# 自動選擇音檔
import os
audio_file = None

# 檢查是否有音檔
if os.path.exists('your_audio.wav'):
    audio_file = 'your_audio.wav'
    print(f"找到音檔: {audio_file}")
else:
    # 創建測試音檔
    print("未找到 'your_audio.wav'，創建測試音檔...")
    audio_file, _ = create_test_audio('test_audio.wav')
    print(f"✓ 已創建測試音檔: {audio_file}")

# 示範用法
try:
    # 調整語速為1.5倍（使用 librosa 方法）
    print("\\n使用 librosa 方法調整語速...")
    audio_1_5x, sr = adjust_audio_speed_librosa(audio_file, speed_factor=1.5)
    
    if audio_1_5x is not None:
        original_audio, _ = librosa.load(audio_file)
        print(f"✓ 成功！")
        print(f"  原始長度: {len(original_audio)/sr:.2f}秒")
        print(f"  調整後長度: {len(audio_1_5x)/sr:.2f}秒")
        print(f"  壓縮比例: {len(audio_1_5x)/len(original_audio):.2f}")
        
        # 播放原始音檔
        print("\\n播放原始音檔:")
        display(Audio(audio_file))
        
        # 播放調整後的音檔
        print("播放1.5倍速音檔:")
        display(Audio(audio_1_5x, rate=sr))
        
        # 儲存調整後的音檔
        output_filename = 'audio_1_5x_speed.wav'
        sf.write(output_filename, audio_1_5x, sr)
        print(f"✓ 已儲存為 '{output_filename}'")
        
        # 顯示波形比較
        print("\\n繪製波形比較圖...")
        plt.figure(figsize=(15, 8))
        
        # 原始音檔波形
        plt.subplot(2, 1, 1)
        time_orig = np.linspace(0, len(original_audio)/sr, len(original_audio))
        plt.plot(time_orig, original_audio)
        plt.title('原始音檔波形')
        plt.xlabel('時間 (秒)')
        plt.ylabel('振幅')
        plt.grid(True)
        
        # 調整後音檔波形
        plt.subplot(2, 1, 2)
        time_new = np.linspace(0, len(audio_1_5x)/sr, len(audio_1_5x))
        plt.plot(time_new, audio_1_5x)
        plt.title('1.5倍速音檔波形')
        plt.xlabel('時間 (秒)')
        plt.ylabel('振幅')
        plt.grid(True)
        
        plt.tight_layout()
        plt.show()
    else:
        print("✗ 處理失敗")
        
except Exception as e:
    print(f"✗ 錯誤: {e}")
    print("請檢查以下項目:")
    print("1. 確保音檔路徑正確")
    print("2. 確保已安裝 librosa 和 soundfile")
    print("3. 嘗試重新執行第一個 cell 安裝套件")

KeyboardInterrupt: 

In [ ]:
# 更多使用範例 - 不同語速的示範
print("=== 多種語速示範 ===")

# 使用之前創建或找到的音檔
input_file = 'test_audio.wav' if os.path.exists('test_audio.wav') else None
if not input_file and os.path.exists('your_audio.wav'):
    input_file = 'your_audio.wav'

if not input_file:
    print("創建新的測試音檔...")
    input_file, _ = create_test_audio('multi_test_audio.wav')
    print(f"✓ 已創建: {input_file}")

# 測試不同語速
speed_factors = [0.5, 0.75, 1.0, 1.25, 1.5, 2.0]
results = {}

print(f"\\n使用音檔: {input_file}")
print("處理不同語速...")

for speed in speed_factors:
    try:
        print(f"處理 {speed}x 語速...", end=" ")
        audio_adjusted, sr = adjust_audio_speed_librosa(input_file, speed_factor=speed)
        
        if audio_adjusted is not None:
            # 儲存結果
            output_file = f"speed_{speed}x.wav"
            sf.write(output_file, audio_adjusted, sr)
            results[speed] = (audio_adjusted, sr, output_file)
            print(f"✓ 長度: {len(audio_adjusted)/sr:.2f}秒")
        else:
            print("✗ 失敗")
            
    except Exception as e:
        print(f"✗ 錯誤: {e}")

# 播放比較（選擇幾個代表性的語速）
if results:
    print("\\n=== 播放比較 ===")
    
    # 播放原始音檔
    print("\\n🎵 原始音檔 (1.0x):")
    display(Audio(input_file))
    
    # 播放不同語速
    demo_speeds = [0.5, 1.5, 2.0]
    for speed in demo_speeds:
        if speed in results:
            audio, sr, filename = results[speed]
            print(f"\\n🎵 {speed}x 語速:")
            display(Audio(audio, rate=sr))

# 視覺化不同語速的波形
if len(results) >= 2:
    print("\\n=== 波形比較圖 ===")
    
    # 載入原始音檔
    original_audio, sr_orig = librosa.load(input_file)
    
    # 選擇要比較的語速
    compare_speeds = [0.5, 1.0, 1.5, 2.0]
    available_speeds = [s for s in compare_speeds if s in results or s == 1.0]
    
    fig, axes = plt.subplots(len(available_speeds), 1, figsize=(15, 3*len(available_speeds)))
    if len(available_speeds) == 1:
        axes = [axes]
    
    for i, speed in enumerate(available_speeds):
        if speed == 1.0:
            # 原始音檔
            time_axis = np.linspace(0, len(original_audio)/sr_orig, len(original_audio))
            axes[i].plot(time_axis, original_audio)
            axes[i].set_title(f'原始音檔 ({speed}x 語速)')
        else:
            # 調整後的音檔
            audio, sr, _ = results[speed]
            time_axis = np.linspace(0, len(audio)/sr, len(audio))
            axes[i].plot(time_axis, audio)
            axes[i].set_title(f'{speed}x 語速')
        
        axes[i].set_xlabel('時間 (秒)')
        axes[i].set_ylabel('振幅')
        axes[i].grid(True, alpha=0.3)
        
        # 限制顯示時間（前3秒）
        axes[i].set_xlim(0, 3)
    
    plt.tight_layout()
    plt.show()

# 總結
print("\\n=== 處理結果總結 ===")
print(f"成功處理 {len(results)} 種語速")
print("\\n已生成的檔案:")
for speed, (audio, sr, filename) in results.items():
    file_size = os.path.getsize(filename) / 1024  # KB
    print(f"  {filename} - {speed}x語速, {len(audio)/sr:.1f}秒, {file_size:.1f}KB")

print("\\n💡 使用技巧:")
print("1. 0.5x - 適合學習語言或聽困難內容")
print("2. 1.25x - 適合日常播客或講座")  
print("3. 1.5x - 適合快速瀏覽或複習")
print("4. 2.0x - 適合熟悉內容的快速播放")

print("\\n✅ 完成！所有音檔都保持了原始音調。")
